In [1]:
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Torch built with CUDA:", torch.backends.cuda.is_built())

Torch version: 2.5.1+cu121
CUDA available: True
CUDA version: 12.1
Torch built with CUDA: True


In [2]:
import llama_cpp

print(llama_cpp.__version__)
print(llama_cpp.llama_cpp._lib)  # biblioteka natywna

0.3.9
<CDLL 'c:\Users\Przemek\Documents\Studia\8_semestr\LCS\lsc-llm-rpg-project\.venv\Lib\site-packages\llama_cpp\lib\llama.dll', handle 7ff9b3630000 at 0x1e8a62faf30>


In [3]:
from llama_cpp import Llama
from llm_rpg.configs import settings

In [4]:
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import init_chat_model

In [5]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [ ]:


template_building = """
Na podstawie poniższego tekstu wypisz trzy informacje, jeśli można je wywnioskować. 
Wypełnij tylko nawiasy kwadratowe — podaj własne, wywnioskowane określenia. 
Jeśli którejś informacji nie da się ustalić, pomiń ją całkowicie (nie wypisuj jej).
Odpowiedzi ustaw w podany sposób, w osobnych linijkach każda, 

Miejsce akcji: [odpowiedź]
Czas akcji: [odpowiedź]
Uniwersum: [odpowiedź]

Teskt do wywnioskowania:
{question}
"""
template_place = """
Scena musi zawierać te informacje:
{place} 
{universe}.
Scena musi nadawać się do fabuły gry RPG Dungeons and Dragons
Opisz nie wielki obszar, opisz scenerię.
Nie generuj instrukcji jak wykonać taki świat.
Nie opisuj bohaterów. Nie powtarzaj się. Podaj tylko kilka zdań treściwych.
Twoja odpowiedź ma składać się jedynie z opisu według schematu:

Opis scenerii:
[tekst]
"""
template_plot = """
Jesteś mistrzem gry RPG.
Wygeneruj tylko jedną wersję, bez powtórzeń
Na podstawie poniższych informacji stwórz **jedną krótką fabułę** oraz jasno zdefiniowany cel przygody.
- Świat: {universe}
- Miejsce akcji: {place}

Twoja odpowiedź ma mieć dokładnie dwie części. Nie powtarzaj sekcji historii ani celu więcej niż raz.
Użyj tylko poniższego formatu:
Historia: [tekst]
Cel: [tekst]
"""

template_exploration = """
Jesteś mistrzem gry RPG.
Na podstawie poniższych informacji stwórz **kilka punktów** które znajduje się w obecnej lokacji i mogą być ciekawe dla graczy:
- Świat: {universe}
- Miejsce akcji: {place}
- Historia: {plot}
- Cel: {quest}

Wymyśl lokację, a następnie kilka misji związanych z tą lokacją.
Jedna misja wokół musi być powiązane z głównym celem misji. Musi być to mały krok do ostatecznego celu.
Wypisz je w liście, jedna pod drugą, tak aby było je łatwo odczytać.
"""


template_exploration_continuation = '''
Jesteś mistrzem gry RPG.
Na podstawie poniższej listy misji:
{current_plot}

Wygeneruj ciąg dalszy historii. Nie powtarzaj się, tamte zdarzenia to już przeszłość.
Przy generowaniu najważniejszym punktem jest ta akcja:
{action}
'''

prompt = PromptTemplate.from_template(template_exploration_continuation)

In [35]:
n_gpu_layers = -1  # The number of layers to put on the GPU. The rest will be on the CPU. If you don't know how many layers there are, you can use -1 to move all to GPU.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path=f"../{settings.models_path}/Bielik-4.5B-v3.0-Instruct.Q8_0.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    verbose=False,  # Verbose is required to pass to the callback manager
    n_ctx=8192
)

In [36]:
llm_chain = prompt | llm
question = "Historia dziejąca się w świecie gwiezdnych wojen za czasów Imperium na pustyniach planety Tatooine. Imperium przejęło włądzę w całej galaktyce za sprawą Dartha Vadera oraz armii"

place = "Tattooine"
time = "Czasy Imperium (lata 0-5 BBY)"

map_desription = '''Opis scenerią:
Planeta Tattooine, położona na Zewnętrznych Rubieżach, jest fascynującym miejscem w uniwersum Gwiezdnych Wojen.
Znajdująca się w systemie o tej samej nazwie, planeta wyróżnia się nie tylko swoim niezwykłym klimatem i geologią, ale także jako ważne miejsce dla wielu postaci i wydarzeń z sagi Gwiezdne Wojny.
Planeta Tattooine 
Gwiezdne wojny (Star wars), czasy Imperium.
Scena musi nadawać się do fabuły gry RPG Dungeons and Dragonsi
Niewielki obszar, scena opisująca:
Wieś Kanopus na planecie Tattooine, położona w dolinie rzeki Amal, otoczona rozległymi pustyniami piaszczystymi i czerwonymi skałami.
Scena przedstawia wieś z perspektywy jej mieszkańców, ukazując życie codzienne wsi, handel, rzemiosło oraz interakcje między mieszkańcami różnych ras i kultur.'''
universe = 'Gwiezdne wojny, Epizod IV: Nowa nadzieja i kolejne filmy z serii'

plot = '''Historia:
W Kanposhu, spokojnej wiosce na planecie Tattooine, mieszkańcy żyją w harmonii z otaczającym ich światem.
Jednak pewnego dnia, podczas rutynowych prac na rzece Amal, mieszkańcy odkryli coś niezwykłego pod powierzchnią dna rzeki. 

Odkryli starożytny artefakt, wykonany ze świetlistej, niebieskiej substancji, która emanowała delikatnym blaskiem nawet w najciemniejszych zakamarkach wioski.

Artefakt ten był niezwykle cenny i tajemniczy, a jego odkrycie wywołało wielkie poruszenie wśród mieszkańców Kanposhu. 
Cel:
Odnalezienie właściciela artefaktu poprzez zbadanie historii wioski oraz poznanie losów jej mieszkańców.
Historia:
W Kanposhu, spokojnej wiosce na planecie Tattooine, mieszkańcy żyją w harmonii z otaczającym ich światem.
Jednak pewnego dnia, podczas rutynowych prac na rzece Amal, mieszkańcy odkryli coś niezwykłego pod powierzchnią dna rzeki. 

Odkryli starożytny artefakt, wykonany ze świetlistej, niebieskiej substancji, która emanowała delikatnym blaskiem nawet w najciemniejszych zakamarkach wioski.

Arte'''

history = '''W Kanposhu, spokojnej wiosce na planecie Tattooine, mieszkańcy żyją w harmonii z otaczającym ich światem.
Jednak pewnego dnia, podczas rutynowych prac na rzece Amal, mieszkańcy odkryli coś niezwykłego pod powierzchnią dna rzeki. 
Odkryli starożytny artefakt, wykonany ze świetlistej, niebieskiej substancji, która emanowała delikatnym blaskiem nawet w najciemniejszych zakamarkach wioski.
Artefakt ten był niezwykle cenny i tajemniczy, a jego odkrycie wywołało wielkie poruszenie wśród mieszkańców Kanposhu. '''

quest = '''Odnalezienie właściciela artefaktu poprzez zbadanie historii wioski oraz poznanie losów jej mieszkańców'''

current_plot = '''Lokacja:
Wieś Kanposhu na planecie Tattooine, położona w dolinie rzeki Amal, otoczona rozległymi pustyniami piaszczystymi i czerwonymi skałmi.
Scena przedstawia wieś z perspektywy jej mieszkańców, ukazując życie codzienne wsi, handel, rzemiosło oraz interakcje między mieszkańcami różnych ras i kultur.
- Misja 1: Zbadanie historii wioski Kanposhu oraz poznanie losów jej mieszkańców w celu odszukania właściciela starożytnego artefaktu znalezionego pod powierzchnią dna rzeki Amal na planecie Tattooine.

- Misja 2: Przepytanie mieszkańców wsi Kanposhu oraz okolicznych osad i miasteczek z planety Tattooine o ich wiedzę na temat starożytnego artefaktu wykopanego pod powierzchnią dna rzeki Amal na planecie Tattooine, a także informacji dotyczących jego właściciela lub miejsca, gdzie mógł zostać ukryty.
- Misja 3: Odszukanie i zebranie wszelkich możliwych informacji na temat starożytnych cywilizacji i kultur zamieszkujących planetę Tattooine w przeszłości oraz ich związków z odnalezionym artefaktem, co może prowadzić do odkrycia tożsamości właściciela starożytnego artefaktu wykopanego pod powierzchnią dna rzeki
'''

action = 'Udajemy się do Gubernatora dostając informację, że posiada on szczegóły pochodzenia artefaktu'
# llm_chain.invoke({"question": question})
# llm_chain.invoke({"place": map_desription, 'universe' : universe})
# llm_chain.invoke({"place": map_desription, 'universe' : universe, 'plot': plot, 'quest': quest})
# llm_chain.invoke({"place": map_desription, 'universe' : universe, 'plot': plot, 'quest': quest, 'current_plot' : current_plot, 'action' : action})
llm_chain.invoke({'current_plot' : current_plot, 'action' : action})





Oto dalszy ciąg historii:
Po otrzymaniu od Gubernatora szczegółów pochodzenia starożytnego artefaktu, bohaterowie postanowili zagłębić się w przeszłość planety Tattooine.
Z pomocą lokalnych historyków i archeologów, bohaterowie odkopywali pozostałości starożytnych cywilizacji, takich jak Jeźdźcy Tusken, Durosjanie czy nawet zaginione plemiona Twilekanów.
W trakcie badań bohaterów ogarniało zdumienie na widok zaawansowanej technologii, jaką starożytni cywilizacje rozwijały na przestrzeni wieków.
Ale wśród wszystkich odkryć i znalezisk bohaterowie nie znaleźli ani właściciela artefaktu, ani jego lokalizacji w przestrzeni.
Po powrocie do Gubernatora bohaterowie opowiedzieli mu o swoich odkryciach, ale bez konkretnej odpowiedzi na pytanie dotyczące pochodzenia artefaktu i jego właściciela.
Gubernator podziękował bohaterom za ich pracę i dodał, że dalsze badania nad starożytnymi cywilizacjami i pochodzeniem artefaktu będą kontynuowane z pomocą lokalnych historyków i archeologów, a także Gub

'Oto dalszy ciąg historii:\nPo otrzymaniu od Gubernatora szczegółów pochodzenia starożytnego artefaktu, bohaterowie postanowili zagłębić się w przeszłość planety Tattooine.\nZ pomocą lokalnych historyków i archeologów, bohaterowie odkopywali pozostałości starożytnych cywilizacji, takich jak Jeźdźcy Tusken, Durosjanie czy nawet zaginione plemiona Twilekanów.\nW trakcie badań bohaterów ogarniało zdumienie na widok zaawansowanej technologii, jaką starożytni cywilizacje rozwijały na przestrzeni wieków.\nAle wśród wszystkich odkryć i znalezisk bohaterowie nie znaleźli ani właściciela artefaktu, ani jego lokalizacji w przestrzeni.\nPo powrocie do Gubernatora bohaterowie opowiedzieli mu o swoich odkryciach, ale bez konkretnej odpowiedzi na pytanie dotyczące pochodzenia artefaktu i jego właściciela.\nGubernator podziękował bohaterom za ich pracę i dodał, że dalsze badania nad starożytnymi cywilizacjami i pochodzeniem artefaktu będą kontynuowane z pomocą lokalnych historyków i archeologów, a ta

In [92]:
split_plot = plot.split('\n')
history_index = None
quest_index = None

history = None
quest = None
for line_idx, line in enumerate(split_plot):
    print(history_index, quest_index, history_index is not None and quest_index is not None)
    if line.lower().find('historia') == 0:
        if history_index is not None and quest_index is not None:
            history = split_plot[history_index+1:quest_index]
            quest = split_plot[quest_index+1:line_idx]
            break
        history_index = line_idx
    elif line.lower().find('cel') == 0:
        quest_index = line_idx

if history is None and history_index is not None and quest_index is not None:
    history = split_plot[history_index:quest_index]
    quest = split_plot[quest_index:line_idx]

history = '\n'.join(filter(lambda x: x, history))
quest = '\n'.join(filter(lambda x: x, quest))
print(history)
print(quest)

None None False
0 None False
0 None False
0 None False
0 None False
0 None False
0 None False
0 None False
0 7 True
0 7 True
W Kanposhu, spokojnej wiosce na planecie Tattooine, mieszkańcy żyją w harmonii z otaczającym ich światem.
Jednak pewnego dnia, podczas rutynowych prac na rzece Amal, mieszkańcy odkryli coś niezwykłego pod powierzchnią dna rzeki. 
Odkryli starożytny artefakt, wykonany ze świetlistej, niebieskiej substancji, która emanowała delikatnym blaskiem nawet w najciemniejszych zakamarkach wioski.
Artefakt ten był niezwykle cenny i tajemniczy, a jego odkrycie wywołało wielkie poruszenie wśród mieszkańców Kanposhu. 
Odnalezienie właściciela artefaktu poprzez zbadanie historii wioski oraz poznanie losów jej mieszkańców.
